# Metric Extraction

Now I need to extract the metrics that I want to measure and check if the have any relation to the convergence speed.

In [36]:
# %pip install numpy
# %pip install pandas
# %pip install tqdm
# %pip install scipy
import os
import time
import math
import pickle
import numpy as np
import pandas as pd
from tqdm import tqdm
from scipy.sparse import dok_matrix

import warnings
warnings.filterwarnings("ignore")

In [37]:
FILENAME = "./matrices_final.csv"

In [38]:
def get_number_of_edges(matrix: dok_matrix) -> int:
    """
    Gets the number of edges in a given matrix.

    :param matrix: matrix to count the edges of
    :return: the number of edges
    """
    return matrix.count_nonzero()

In [39]:
def get_number_of_nodes(matrix: dok_matrix) -> int:
    """
    Gets the number of nodes (states) in a given matrix.

    :param matrix: matrix to count the nodes of
    :return: the number of nodes
    """
    return matrix.get_shape()[0] ** 2

In [40]:
def get_in_degree(degree_dict: dict, n: int) -> tuple:
    """
    Calculate the average and maximum in-degree of a given matrix.

    :param degree_dict: in degree dictionary
    :param n: number of states o the matrix
    :return: a tuple - (average, max), in-degree of the matrix
    """
    total_sum = 0
    maximum = 0
    for key in degree_dict:
        total_sum += degree_dict[key]
        maximum = max(degree_dict[key], maximum)
    return (total_sum / n), maximum

In [41]:
def get_out_degree(degree_dict: dict, n: int) -> tuple:
    """
    Calculate the average and maximum out-degree of a given matrix.

    :param degree_dict: out degree dictionary
    :param n: number of states o the matrix
    :return: a tuple - (average, max), out-degree of the matrix
    """
    total_sum = 0
    maximum = 0
    for key in degree_dict:
        total_sum += degree_dict[key]
        maximum = max(degree_dict[key], maximum)
    return (total_sum / n), maximum

In [42]:
import math
def get_sle(matrix: dok_matrix) -> int:
    """
    Get the second largest eigenvalue for a given matrix. We take the second largest because the largest is 1 for all of them and this does not give us any information. The second largest will also be the slowest to converge compared to the rest.

    :param matrix: matrix to get the eigenvalue of
    :return: the second largest eigenvalue
    """
    m = matrix.toarray()
    if np.isnan(m).any():
        print("Has NaN")
        print(matrix.shape)
        for i in range(len(m)):
            for j in range(len(m)):
                if math.isnan(m[i][j]):
                    print(f'[{i}, {j}] = {m[i][j]}')
    if np.isinf(m).any():
        print("Has Inf")
        print(m)
    values, vectors = np.linalg.eig(m)
    values.sort()
    return values[-2]

In [43]:
def get_diameter_radius(from_to: dict, n: int) -> tuple:
    """
    Get the diameter and the radius of a matrix. The eccentricity is the longest hopcount between a node and any other node. The diameter is the largest eccentricity and the radius is the smallest eccentricity.

    :param from_to: dictionary with transitions
    :param n: number of states o the matrix
    :return: (diameter, radius)
    """
    eccentricity = []
    for start_node in range(n):
        max_hopcount = 0
        for destination_node in range(n):
            hop_groups = [from_to[start_node]]
            found = False
            while not found:
                if destination_node in hop_groups[-1]:
                    max_hopcount = max(max_hopcount, len(hop_groups))
                    found = True
                else:
                    hop_group = set()
                    for node in hop_groups[-1]:
                        hop_group.update(from_to[node])
                    hop_groups.append(list(hop_group))
        eccentricity.append(max_hopcount)
    return max(eccentricity), min(eccentricity)

In [44]:
def get_necessary_metrics(matrix: dok_matrix, n: int):
    """
    Helping function. Reduces the times that I need to loop through the matrix.

    :param matrix: to get the metrics for
    :param n: number of states
    :return:
    """
    from_to = dict()
    out_degree_dict = dict()
    in_degree_dict = dict()
    for i in range(n):
        for j in range(n):
            if matrix[i, j] != 0:
                if i in from_to:
                    from_to[i].append(j)
                else:
                    from_to[i] = [j]
                if j in out_degree_dict:
                    out_degree_dict[j] += 1
                else:
                    out_degree_dict[j] = 1
                if i in in_degree_dict:
                    in_degree_dict[i] += 1
                else:
                    in_degree_dict[i] = 1

    return from_to, in_degree_dict, out_degree_dict

In [45]:
def calculate_dif(vector1: list, vector2: list) -> float:
    """
    Calculate the absolute difference between the two vectors.

    :param vector1:
    :param vector2:
    :return:
    """
    sum = 0
    for i in range(len(vector1)):
        sum += abs(vector2[i] - vector1[i])
    return sum

In [46]:
def get_index(vector: list, val: complex) -> int:
    """
    Get the index of a value in a vector. Rounding after 15 decimals because of rounding error.

    :param vector:
    :param val:
    :return: the index of the value in the vector or -1 if not found
    """
    for i in range(len(vector)):
        if round(vector[i].real, 15) == val.real and vector[i].imag == val.imag:
            return i
    return 0
    return -1

In [47]:
def get_convergence_speed(matrix: dok_matrix, n: int) -> float:
    """
    Calculate the convergence speed of the matrix.

    :param matrix: matrix to calculate the convergence speed for
    :param n: number of states of the matrix
    :return: the convergence speed, iteration count
    """

    epsilon = 1 * (10 ** -4)

    vector = np.zeros(n)
    vector[0] = 1

    next_vector = vector * matrix
    starting_time = time.time()
    iter_count = 1
    while calculate_dif(vector, next_vector) >= epsilon:
        vector, next_vector = next_vector, next_vector * matrix
        iter_count += 1
    # print(vector)
    # print(next_vector)
    ending_time = time.time()
    return ending_time - starting_time, iter_count

In [48]:
# matrix = dok_matrix([[0, 1, 0], [0, 1/2, 1/2], [1/2, 0, 1/2]])
# print(get_convergence_speed(matrix, 3))

In [49]:
def get_all_matrix_data(matrix: dok_matrix) -> list:
    n = matrix.get_shape()[0]
    from_to, in_degree_dict, out_degree_dict = get_necessary_metrics(matrix, n)
    row = []
    row.append(f"{n}x{n}")
    row.append(get_number_of_nodes(matrix))
    row.append(get_number_of_edges(matrix))
    diameter, radius = get_diameter_radius(from_to, n)
    row.append(diameter)
    row.append(radius)
    average_in, max_in = get_in_degree(in_degree_dict, n)
    average_out, max_out = get_out_degree(out_degree_dict, n)
    row.append(average_in)
    row.append(max_in)
    row.append(max_out)
    sle = get_sle(matrix)
    row.append(sle)
    num = complex(sle)
    row.append(math.sqrt(num.real ** 2 + num.imag ** 2))
    time, iterations = get_convergence_speed(matrix, n)
    row.append(time)
    row.append(iterations)
    return row

In [50]:
# sle = second largest eigenvalue
COLUMNS = ['matrix', 'number_of_nodes', 'number_of_edges', 'diameter', 'radius',
           'average_deg', 'max_in_deg', 'max_out_deg',
           'sle', 'norm_sle', 'convergence_speed', 'convergence_iterations']
data = pd.DataFrame(columns=COLUMNS)
DIR = './data/generated'

for filename in tqdm(os.listdir(DIR)):
    f = os.path.join(DIR, filename)

    if os.path.isfile(f):
        with open(f, "rb") as file:
            matrices = pickle.load(file)
            for matrix in matrices:
                row = get_all_matrix_data(matrix)
                new_data = pd.DataFrame(data=[row], columns=COLUMNS)
                data = pd.concat([data, new_data], ignore_index=True)

data.to_csv(FILENAME)

100%|██████████| 190/190 [16:52<00:00,  5.33s/it]


In [51]:
data.tail(10)

,matrix,number_of_nodes,number_of_edges,diameter,radius,average_deg,max_in_deg,max_out_deg,sle,norm_sle,convergence_speed,convergence_iterations
1890,99x99,9801,7425,2,2,75.0,75,85,0.091393+0.014212j,0.092491,0.021090,6
1891,99x99,9801,6930,2,2,70.0,70,79,0.101664+0.000000j,0.101664,0.017760,6
1892,99x99,9801,6435,2,2,65.0,65,75,0.109516+0.000000j,0.109516,0.017045,6
1893,99x99,9801,5940,2,2,60.0,60,72,0.108200+0.000000j,0.108200,0.017086,7
1894,99x99,9801,5445,2,2,55.0,55,70,0.117997+0.020878j,0.119830,0.015735,7
1895,99x99,9801,4950,2,2,50.0,50,63,0.121992+0.038061j,0.127792,0.015039,7
1896,99x99,9801,4455,2,2,45.0,45,56,0.143682+0.000000j,0.143682,0.013547,7
1897,99x99,9801,3960,2,2,40.0,40,51,0.143744+0.003189j,0.143780,0.012963,7
1898,99x99,9801,3465,2,2,35.0,35,47,0.170048+0.012544j,0.170510,0.011000,8
1899,99x99,9801,2970,2,2,30.0,30,45,0.181490+0.031944j,0.184280,0.010600,8


In [52]:
data.to_csv('./matrices_final.csv')

In [53]:
sdasfkjlds

NameError: name 'sdasfkjlds' is not defined

## Extract the metrics for the downloaded files

In [ ]:
def is_ergodic(matrix, n):
    """
    An ergodic matrix is aperiodic and irreducible. By Wielandt's theorem if when the matrix is multiplied by itself m
    times, where m = (n - 1) * (n - 1) + 1, and all its entries are positive then the matrix is ergodic. n is the number
    of sates.

    :param matrix: matrix to check
    :return: true if the matrix is ergodic, false otherwise
    """
    matrix = matrix.tocsr(copy=True)
    m = (n - 1) * (n - 1) + 1
    multiplicities = [matrix]
    for i in range(int(math.log(m, 2))):
        matrix = matrix.dot(matrix)
        multiplicities.append(matrix)
    index = len(multiplicities) - 1
    res = None
    while m > 0:
        if m & 1:
            if res is None:
                res = multiplicities[index]
            else:
                res = res.dot(multiplicities[index])
        index -= 1
        m = m >> 1
    return res.count_nonzero() == n * n

In [ ]:
COLUMNS_DOWNLOAD = ['matrix_name', 'number_of_nodes', 'number_of_edges', 'diameter', 'radius', 'average_in_deg', 'average_out_deg', 'max_in_deg', 'max_out_deg', 'sle']
downloaded_data = pd.DataFrame(columns=COLUMNS_DOWNLOAD)
DIR_DOWNLOAD = './data/downloaded'

i = 1
for filename in os.listdir(DIR_DOWNLOAD):
    f = os.path.join(DIR_DOWNLOAD, filename)
    if not filename.endswith('.pickle'):
        continue
    if os.path.isfile(f):
        print(f"{i}: {filename}")
        i += 1
        with open(f, "rb") as file:
            try:
                matrix = pickle.load(file)
                n = matrix.get_shape()[0]
                if not is_ergodic(matrix, n):
                    print("Not Ergodic")
                    continue
                row = get_all_matrix_data(matrix)
            except MemoryError:
                print(f"File {filename}'s matrix is too big ({n}x{n})")
                continue
            row[0] = filename
            new_data = pd.DataFrame(data=[row], columns=COLUMNS_DOWNLOAD)
            downloaded_data = pd.concat([downloaded_data, new_data], ignore_index=True)
downloaded_data.to_csv("downloaded_with_metrics.csv")

In [ ]:
downloaded_data = downloaded_data.sort_values(by=['number_of_nodes'], ignore_index=True)
downloaded_data.head()

In [ ]:
downloaded_data.tail()

In [ ]:
downloaded_data.to_csv("downloaded_with_metrics.csv")